In [1]:
from notears.locally_connected import LocallyConnected
from notears.lbfgsb_scipy import LBFGSBScipy
from notears.trace_expm import trace_expm
import torch
import torch.nn as nn
import numpy as np
import math

from notears.locally_connected import LocallyConnected
from notears.lbfgsb_scipy import LBFGSBScipy
from notears.trace_expm import trace_expm
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import math
import notears.utils as ut
import cdt
from metric import MetricsDAG
from pgmpy.models import BayesianModel
import networkx as nx
from sklearn.model_selection import KFold
from catboost import Pool, CatBoostRegressor
from sklearn.metrics import mean_squared_error
from ax.service.managed_loop import optimize


No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.


In [2]:
from ax.service.managed_loop import optimize


In [3]:
data, graph = cdt.data.load_dataset('sachs')


In [4]:
len(data.columns)

11

In [16]:
from notears.nonlinear import NotearsMLP, notears_nonlinear


In [26]:
def train_on_saches(train, parameterization):
    torch.set_default_dtype(torch.double)
    np.set_printoptions(precision=3)
    ut.set_random_seed(123)

    d= 11 # TODO Needs to change 
    model = NotearsMLP(dims=[d, 10, 1], bias=True)
    global W_est
    W_est = notears_nonlinear(model,
                          train.to_numpy(), # Change to train
                          lambda1=parameterization['lambda1'],
                          lambda2=parameterization['lambda2'],
                          max_iter=100,
                          h_tol=1e-8,
                          rho_max=1e+16,
                          w_threshold=parameterization['w_threshold']
    )
    W_est_binary = W_est !=0
    assert ut.is_dag(W_est)
    return W_est_binary

In [27]:
def train_evaluate_given_parameter_config(parameterization: dict):
    print('In the loop')
    kfold = KFold(3, True, 1)
    folds_score = []
    for train_idx, test_idx in kfold.split(data):
        train, test = data.iloc[train_idx, :], data.iloc[test_idx, :]

        # 1. Find the estiamte dgraph using inner fold training data
        weight_estimated = train_on_saches(train, parameterization)
        sm = nx.convert_matrix.from_numpy_array(W_est != 0)

        bn = BayesianModel()
        bn.add_edges_from(sm.edges)
        nodes = list(bn.nodes())
        # 2. For each node, train a regression tree model and record its performance
        score_list = []
        for node in nodes:
            markov_blanket = bn.get_markov_blanket(node)
            
            train_x, train_y = train.loc[:, markov_blanket], train.loc[:, node]
            # Normalize train_x, train_y, test_x, test_y by scale of the training dataset
            train_x=(train_x-train_x.mean())/train_x.std()
            train_y=(train_y-train_y.mean())/train_y.std()
            
            test_x, test_y = test.loc[:, markov_blanket], test.loc[:, node]
            test_x=(test_x-train_x.mean())/train_x.std()
            test_y=(test_y-train_y.mean())/train_y.std()
            
            train_pool = Pool(train_x, train_y)

            model = CatBoostRegressor(iterations=50, verbose=False)
            model.fit(train_pool)
            mse = mean_squared_error(model.predict(test_x), test_y)
            print(f'MSE at Node {node} is = {mse}')
            score_list.append(mse)
        # 3. Average fold score of average score across nodes
        fold_score = np.mean(score_list)
        folds_score.append(fold_score)
    avg_folds_score = np.mean(folds_score)
    return avg_folds_score


In [28]:
def bayesian_optimize(data):
    best_parameters, best_values, experiment, model = optimize(
            parameters=[
              {
                "name": "lambda1",
                "type": "range",
                "bounds": [0.01, 0.5],
                "log_scale": True
              },
              {
                 "name": "lambda2",
                 "type": "range",
                 "bounds": [0.01, 0.5],
                 "log_scale": True
              },
              {
                  "name": "w_threshold",
                  "type": "range",
                  "bounds": [0.01, 0.5],
                  "log_scale": True
               }
            ],
            # Booth function
            evaluation_function=train_evaluate_given_parameter_config,
            minimize=True,
        )
    return best_parameters, best_values, experiment, model

In [29]:
best_parameters, values, experiment, model  = bayesian_optimize(data)

[INFO 12-02 13:14:19] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter lambda1. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 12-02 13:14:19] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter lambda2. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 12-02 13:14:19] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter w_threshold. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 12-02 13:14:19] ax.modelbridge.dispatch_utils: Using Bayesian optimization since there are more ordered parameters than there are categories for the unordered categorical parameters.
[INFO 12-02 13:14:19] ax.modelbridge.dispatch_utils

In the loop


[ERROR 12-02 14:26:37] ax.service.managed_loop: Encountered exception during optimization: 
Traceback (most recent call last):
  File "C:\Users\sheng\anaconda3\envs\causalnex\lib\site-packages\ax\service\managed_loop.py", line 229, in full_run
    self.run_trial()
  File "C:\Users\sheng\anaconda3\envs\causalnex\lib\site-packages\ax\utils\common\executils.py", line 147, in actual_wrapper
    return func(*args, **kwargs)
  File "C:\Users\sheng\anaconda3\envs\causalnex\lib\site-packages\ax\service\managed_loop.py", line 207, in run_trial
    raw_data={
  File "C:\Users\sheng\anaconda3\envs\causalnex\lib\site-packages\ax\service\managed_loop.py", line 208, in <dictcomp>
    arm.name: self._call_evaluation_function(arm.parameters, weight)
  File "C:\Users\sheng\anaconda3\envs\causalnex\lib\site-packages\ax\service\managed_loop.py", line 147, in _call_evaluation_function
    evaluation = self.evaluation_function(parameterization)
  File "C:\Users\sheng\AppData\Local\Temp/ipykernel_40560/6514

ValueError: Cannot identify best point if experiment contains no data.

In [114]:
best_parameters

{'lambda1': 0.13347911283786074,
 'lambda2': 0.01,
 'w_threshold': 0.025991817570101477}

In [62]:
from notears.nonlinear import NotearsMLP, notears_nonlinear


def train_on_saches(train, parameterization):
    torch.set_default_dtype(torch.double)
    np.set_printoptions(precision=3)
    ut.set_random_seed(123)

    d= 10
    model = NotearsMLP(dims=[d, 10, 1], bias=True)
    global W_est
    W_est = notears_nonlinear(model,
                          train.to_numpy(), # Change to train
                          lambda1=parameterization['lambda1'],
                          lambda2=parameterization['lambda2'],
                          max_iter=100,
                          h_tol=1e-8,
                          rho_max=1e+16,
                          w_threshold=parameterization['w_threshold']
    )
    W_est_binary = W_est !=0
    assert ut.is_dag(W_est)
    return W_est_binary

In [63]:
data = pd.DataFrame(X)

In [76]:
parameterization = {'lambda1': 0.1, 'lambda2': 0.1, 'w_threshold': 0.5}
# prepare cross validation
kfold = KFold(3, True, 1)
folds_score = []
# enumerate splits
for train_idx, test_idx in kfold.split(data):
    train, test = data.iloc[train_idx, :], data.iloc[test_idx, :]

    # 1. Find the estiamte dgraph using inner fold training data
    weight_estimated = train_on_saches(train, parameterization)
    sm = nx.convert_matrix.from_numpy_array(W_est != 0)

    bn = BayesianModel()
    bn.add_edges_from(sm.edges)
    nodes = list(bn.nodes())
    # 2. For each node, train a regression tree model and record its performance
    score_list = []
    for node in nodes:
        markov_blanket = bn.get_markov_blanket(node)
        train_x, train_y = train.loc[:, markov_blanket], train.loc[:, node]
        test_x, test_y = test.loc[:, markov_blanket], test.loc[:, node]
        train_pool = Pool(train_x, train_y)
        
        model = CatBoostRegressor(iterations=50, verbose=False)
        model.fit(train_pool)
        mse = mean_squared_error(model.predict(test_x), test_y)
        print(f'MSE at Node {node} is = {mse}')
        score_list.append(mse)
    # 3. Average fold score of average score across nodes
    fold_score = np.mean(score_list)
    folds_score.append(fold_score)
avg_folds_score = np.mean(folds_score)

Pass shuffle=True, random_state=1 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
BayesianModel has been renamed to BayesianNetwork. Please use BayesianNewtork class, BayesianModel will be removed in future.


MSE at Node 0 is = 1.0002795693903483
MSE at Node 5 is = 2.642699904085316
MSE at Node 1 is = 0.14025862376885628
MSE at Node 6 is = 0.9136642209257972
MSE at Node 2 is = 2.337938271621135
MSE at Node 8 is = 1.1136731254378014
MSE at Node 3 is = 2.7494904881783127
MSE at Node 4 is = 5.792277352579166
MSE at Node 7 is = 3.704561191788133


BayesianModel has been renamed to BayesianNetwork. Please use BayesianNewtork class, BayesianModel will be removed in future.


MSE at Node 0 is = 1.2664815966612395
MSE at Node 5 is = 6.3541927987466815
MSE at Node 1 is = 0.13216892032337338
MSE at Node 6 is = 0.7887898931635486
MSE at Node 2 is = 2.3806958713255253
MSE at Node 8 is = 1.4247255658673081
MSE at Node 3 is = 4.627122627229899
MSE at Node 4 is = 5.243977437147302
MSE at Node 7 is = 6.4353894234641835


BayesianModel has been renamed to BayesianNetwork. Please use BayesianNewtork class, BayesianModel will be removed in future.


MSE at Node 0 is = 1.8170542200009754
MSE at Node 5 is = 9.628422426190236
MSE at Node 1 is = 0.14348376869863189
MSE at Node 6 is = 1.3764157380060549
MSE at Node 2 is = 3.13248662628116
MSE at Node 4 is = 4.187287071733469
MSE at Node 8 is = 1.478422452018934
MSE at Node 3 is = 10.730524498046723
MSE at Node 7 is = 4.241254852045011


In [77]:
avg_folds_score

3.1771755012861163

In [71]:
range(train.shape[1])

range(0, 10)

In [75]:
nodes

[0, 5, 1, 6, 2, 8, 3, 4, 7]

In [72]:
bn.get_markov_blanket(nodes[node_idx])

IndexError: list index out of range

In [56]:
%%time
from sklearn.metrics import mean_squared_error
train_x, train_y = train.loc[:, markov_blanket], train.loc[:, nodes[node_idx]]
test_x, test_y = test.loc[:, markov_blanket], test.loc[:, nodes[node_idx]]
train_pool = Pool(train_x, train_y)

model = CatBoostRegressor(iterations = 100, verbose=False)
model.fit(train_pool)
score_list.append(mean_squared_error(model.predict(test_x), test_y))

Wall time: 101 ms


In [57]:
mean_squared_error(model.predict(test_x), test_y)

1.0349892143886394

In [42]:
train.loc[:, markov_blanket]

,3,4,5
0,11.207474,6.464077,2.092471
1,11.458089,5.934889,2.098876
2,7.796108,7.890693,5.622521
3,10.808592,9.925525,-0.056029
6,10.083932,13.307149,-1.235286
...,...,...,...
191,11.610377,11.211398,2.923260
192,14.556498,10.419711,4.606644
195,9.852090,5.261232,1.996898
196,-0.872462,0.761941,-3.841677
